# **1) Import the Modules**

Modules are code libraries that contain a set of ready-to-use functions.

* The `ee` module allows developers to interact with Google Earth Engine using the Python programming language.
* The `os` module provides functions to perform tasks such as file and directory operations, process management, and environment variable manipulation.
* The `sys` module facilitates interaction with and manipulation of the Python runtime environment's system-specific parameters and functions.
* The `math` module provides a collection of mathematical functions and constants for performing mathematical operations.
* The `geemap` module allows interactive analysis and visualization of GEE datasets in a Jupyter environment.
* The `matplotlib.pyplot` module provides a collection of functions for creating and customizing plots, diagrams and visualizations.
* The `google.colab` module provides access to some of the unique features and functionality of Google Colab.

In [ ]:
!pip install geemap
!pip install ffmpeg-python

In [ ]:
import ee
import os
import sys
import math
import geemap
import ffmpeg

import matplotlib.pyplot as plt

from google.colab import drive

***Update the geemap package***

If you encounter errors with this notebook, please remove the comment in the line below to update the geemap package to the latest version from GitHub. Restart the Kernel (Menu -> Kernel -> Restart) to take effect.

In [ ]:
# geemap.update_package()

# **2) Authentication Procedure**

This section provides instructions for setting up the Google Earth Engine Python API on Colab and for setting up Google Drive on Colab. These steps should be performed each time you start/restart/rollback a Colab session.

## **2.1) GEE**

The `ee.Authenticate` function authenticates access to the Google Earth Engine servers, while the `ee.Initialize` function initializes it. After executing the following cell, the user is prompted to grant Google Earth Engine access to their Google account.

**Note:** The Earth Engine API is installed by default in Google Colaboratory.

In [ ]:
ee.Authenticate()
ee.Initialize(project="...")

## **2.2) GD**

The `drive.mount` function allows access to specific folders of Google Drive. Granting access to Google Drive allows code running in the notebook to modify files in Google Drive.

**Note:** When using the `Mount Drive` button in the file browser, no authentication codes are required for notebooks edited only by the current user.

In [ ]:
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# **3) Functions**

Data Processing

In [ ]:
sys.path.append(".../utilities")
import S1

Data Visualization

In [ ]:
def calculate_histogram(image, bandName, region):
  """
  Description:
    Calculates the histogram for a specified band of an Earth Engine image within a given region.

  Arguments:
      image (ee.Image): The Earth Engine image for which to calculate the histogram.
      bandName (str): The name of the band for which to compute the histogram.
      region (ee.Geometry): The region of interest (ROI) where the histogram is computed.

  Returns:
      A GEE dictionary containing the histogram values for the specified band within the given region.
  """
  return image.select(bandName).reduceRegion(**{
    "reducer": ee.Reducer.autoHistogram(),
    "geometry": region,
    "scale": 10,
    "maxPixels": 1e9
  }).get(bandName)


def plot_histograms(histograms, plotTitle, destinationPath):
  """
  Description:
    Creates a histogram plot from a dictionary and saves the plot to a file. The dictionary
    is a list of tuples, where each tuple contains two elements: a value and
    its corresponding frequency in the histogram.

  Arguments:
  histograms (dict): A dictionary containing histogram data for each identifier.
  destinationPath (str): The path where the plot will be saved.

  Returns:
    None
  """
  hexValues = {"pre_event": "#deb887", "post_event": "#45b6fe"}

  # Create a figure and a set of subplots
  fig, ax = plt.subplots(figsize=(10, 8))

  # Set plot title
  plt.title(plotTitle, fontsize=18, fontweight="bold", color="#595959")

  # Edit axes

  plt.xticks(fontsize=14, fontweight="bold", color="#595959")
  plt.yticks(fontsize=14, fontweight="bold", color="#595959")

  # Add grid lines
  plt.grid(True, linestyle="--", color="white")

  # Set facecolor
  ax.set_facecolor("lightgray")

  # Set limits
  plt.xlim(-30, 10)

  # Create bar plots
  for key, value in histograms.items():
    x = [item[0] for item in value]
    y = [item[1] for item in value]
    plt.bar(x, y, alpha=0.5, width=0.5, edgecolor="white", color=hexValues[key], label=key)

  # Set legend
  legend = plt.legend()

  for text in legend.get_texts():
    text.set_fontweight("bold")

  # Save plot
  plt.savefig(destinationPath, dpi=500, bbox_inches="tight")
  plt.close()

# **4) Parameters**

In [ ]:
# `Digital Elevation Model`
demProvider = "USGS"

# `Sentinel-1 GRD`
b1 = "VV"
b2 = "VH"

chartIdentifiers = {
  "pre_event": "003747_00476D_87B6",
  "post_event": "004447_005723_9D4B"
}

rasterIdentifiers = {
  "pre_event": "S1A_IW_GRDH_1SDV_20141216T163133_20141216T163158_003747_00476D_87B6",
  "post_event": "S1A_IW_GRDH_1SDV_20150202T163132_20150202T163157_004447_005723_9D4B"
}

# Projection of interest.
projectionCRS = "EPSG:4326"
projectionScale = 10

# Runtime stuff.
bufferDistance = -10000

coordinates = [
  20.058040632927096, 38.102793131409605,
  22.899790328988313, 38.49643039052389,
  22.90055614340465,  38.512305042085146,
  22.637037745648232, 39.82427938096713,
  22.59494267298734,  39.989073630192436,
  22.59136250228799,  39.998627426959494,
  19.688106510271055, 39.606279267839795
]

undesiredAreas = [
  ee.FeatureCollection("...")
]

# GD paths.
destinationFolder = "..."

# **5) Configuration**

In [ ]:
# `Digital Elevation Models`
demConfigs = {
  "CGIAR": {    # `SRTM Digital Elevation Data Version 4`
    "name": "CGIAR/SRTM90_V4",
    "resolution": 90
  },
  "NASA": {     # `NASA NASADEM Digital Elevation`
    "name": "NASA/NASADEM_HGT/001",
    "resolution": 30
  },
  "USGS": {     # `NASA SRTM Digital Elevation`
    "name": "USGS/SRTMGL1_003",
    "resolution": 30
  },
  "ASTER": {    # `AG100: ASTER Global Emissivity Dataset 100-meter V003`
    "name": "NASA/ASTER_GED/AG100_003"
  }
}

# `Sentinel-1 GRD`
s1Config = {
  "name": "COPERNICUS/S1_GRD",
  "mode": "IW",
  "resolution": 10,
  "bands": ["VV", "VH"]
}

sarBluishVisualization = {
  "min": [-18, -25, 1],
  "max": [0, -5, 12],
  "bands": ["VV", "VH", "VVVHD"]
}

sarReddishVisualization = {
  "min": [-25, -25, 0],
  "max": [0, 0, 2],
  "bands": ["VV", "VH", "VVVHQ"]
}

bandCombinations = {
  "sum": {
    "expression": "b1 + b2",
    "name": "b1b2S"
  },
  "difference": {
    "expression": "b1 - b2",
    "name": "b1b2D"
  },
  "product": {
    "expression": "b1 * b2",
    "name": "b1b2P"
  },
  "quotient": {
    "expression": "b1 / b2",
    "name": "b1b2Q"
  },
  "ndpi": {
    "expression": "(b1 - b2) / (b1 + b2)",
    "name": "NDPI"
  }
}

In [ ]:
# Retrieve configurations.
demConfig = demConfigs[demProvider]

# Retrieve GEE assets.
areaOfInterest = ee.Geometry.Polygon(coordinates).buffer(bufferDistance, 1)
ring = ee.Geometry.LinearRing(areaOfInterest.coordinates().flatten())
undesiredAreas = ee.FeatureCollection(undesiredAreas).flatten()
borderVectors = ee.FeatureCollection("FAO/GAUL/2015/level2")

# **6) Data Processing**

Define projection of interest.

In [ ]:
projection = ee.Projection(projectionCRS).atScale(projectionScale)

Load, filter and process raster collections.

In [ ]:
# `Digital Elevation`
elevation = ee.Image(demConfig["name"]).select("elevation").unmask()

# `Sentinel-1 GRD`
preEventRaster = ee.Image("/".join([s1Config["name"], rasterIdentifiers["pre_event"]]))
postEventRaster = ee.Image("/".join([s1Config["name"], rasterIdentifiers["post_event"]]))

# Apply angular-based radiometric slope correction.
preEventRaster = S1.slope_correction(preEventRaster, elevation)
postEventRaster = S1.slope_correction(postEventRaster, elevation)

# Apply a Refined-Lee speckle filter to reduce noise.
preEventRaster = ee.Image(S1.refined_lee(preEventRaster).copyProperties(preEventRaster))
postEventRaster = ee.Image(S1.refined_lee(postEventRaster).copyProperties(postEventRaster))

Handle geometries.

In [ ]:
preEventRaster = preEventRaster.clip(areaOfInterest)
postEventRaster = postEventRaster.clip(areaOfInterest)

region = areaOfInterest.difference(undesiredAreas.geometry(), 1)

Transform vector collections into raster representations.

In [ ]:
undesiredSurfaces = ee.ImageCollection([
  ee.Image.constant(0).toInt().clip(areaOfInterest),
  ee.Image.constant(1).toInt().clipToCollection(borderVectors),
  ee.Image.constant(0).toInt().clipToCollection(undesiredAreas),
])                                                \
.mosaic()                                         \
.clip(areaOfInterest)                             \
.reproject(projection)

Engineer additional raster bands.

In [ ]:
differenceExpression = bandCombinations["difference"]["expression"]
quotientExpression = bandCombinations["quotient"]["expression"]
ndpiExpression = bandCombinations["ndpi"]["expression"]

differenceName = bandCombinations["difference"]["name"]
quotientName = bandCombinations["quotient"]["name"]

In [ ]:
# `PRE_VVVHD`
PRE_VVVHD = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": preEventRaster.select(b1),
      "b2": preEventRaster.select(b2)
    }
  }).rename(differenceName.replace("b1", b1).replace("b2", b2))

# `POST_VVVHD`
POST_VVVHD = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": postEventRaster.select(b2)
    }
  }).rename(differenceName.replace("b1", b1).replace("b2", b2))

# `VVVVD`
VVVVD = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": preEventRaster.select(b1)
    }
  }).rename(differenceName.replace("b1", b1).replace("b2", b1))

# `VHVHD`
VHVHD = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": postEventRaster.select(b2),
      "b2": preEventRaster.select(b2)
    }
  }).rename(differenceName.replace("b1", b2).replace("b2", b2))

# `VVVHD`
VVVHD = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": preEventRaster.select(b2)
    }
  }).rename(differenceName.replace("b1", b1).replace("b2", b2))

# `PRE_VVVHQ`
PRE_VVVHQ = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": preEventRaster.select(b1),
      "b2": preEventRaster.select(b2)
    }
  }).rename(quotientName.replace("b1", b1).replace("b2", b2))

# `POST_VVVHQ`
POST_VVVHQ = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": postEventRaster.select(b2)
    }
  }).rename(quotientName.replace("b1", b1).replace("b2", b2))

# `VVVVQ`
VVVVQ = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": preEventRaster.select(b1)
    }
  }).rename(quotientName.replace("b1", b1).replace("b2", b1))

# `VHVHQ`
VHVHQ = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": postEventRaster.select(b2),
      "b2": preEventRaster.select(b2)
    }
  }).rename(quotientName.replace("b1", b2).replace("b2", b2))

# `VVVHQ`
VVVHQ = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": preEventRaster.select(b2)
    }
  }).rename(quotientName.replace("b1", b1).replace("b2", b2))

# `PRE_NDPI`
preNDPI = ee.Image().expression(**{
    "expression": ndpiExpression,
    "opt_map": {
      "b1": preEventRaster.select(b1),
      "b2": preEventRaster.select(b2)
    }
  }).rename("PRE_NDPI")

# `POST_NDPI`
postNDPI = ee.Image().expression(**{
    "expression": ndpiExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": postEventRaster.select(b2)
    }
  }).rename("POST_NDPI")

# `NDPID`
NDPID = postNDPI.subtract(preNDPI).rename("NDPID")

# `NDPIQ`
NDPIQ = postNDPI.divide(preNDPI).rename("NDPIQ")

# Incorporate extra bands into both pre- & post- event rasters.
preEventRaster = preEventRaster.addBands([PRE_VVVHD, PRE_VVVHQ])
postEventRaster = postEventRaster.addBands([POST_VVVHD, POST_VVVHQ])

# **7) Data Visualization**

Create histograms to evaluate the distribution of band values.

In [ ]:
vvRaster = ee.ImageCollection([preEventRaster.select(b1), postEventRaster.select(b1)]).toBands()  \
  .rename([chartIdentifiers["pre_event"], chartIdentifiers["post_event"]])                        \
  .updateMask(undesiredSurfaces)

vhRaster = ee.ImageCollection([preEventRaster.select(b2), postEventRaster.select(b2)]).toBands()  \
  .rename([chartIdentifiers["pre_event"], chartIdentifiers["post_event"]])                        \
  .updateMask(undesiredSurfaces)

# Calculate total pixel count.
pixelsCount = ee.Image.constant(1).clip(areaOfInterest)   \
  .setDefaultProjection(projection)                       \
  .reduceRegion(**{
    "reducer": ee.Reducer.count(),
    "geometry": areaOfInterest,
    "scale": projectionScale,
    "maxPixels": 1e13
  })

# Calculate mean values.
prevvMean = vvRaster.select(chartIdentifiers["pre_event"]).reduceRegion(**{
    "reducer": ee.Reducer.mean(),
    "geometry": areaOfInterest,
    "scale": projectionScale,
    "maxPixels": 1e13
  }) \
  .get(chartIdentifiers["pre_event"])

prevhMean = vhRaster.select(chartIdentifiers["pre_event"]).reduceRegion(**{
    "reducer": ee.Reducer.mean(),
    "geometry": areaOfInterest,
    "scale": projectionScale,
    "maxPixels": 1e13
  }) \
  .get(chartIdentifiers["pre_event"])

postvvMean = vvRaster.select(chartIdentifiers["post_event"]).reduceRegion(**{
    "reducer": ee.Reducer.mean(),
    "geometry": areaOfInterest,
    "scale": projectionScale,
    "maxPixels": 1e13
  }) \
  .get(chartIdentifiers["post_event"])

postvhMean = vhRaster.select(chartIdentifiers["post_event"]).reduceRegion(**{
    "reducer": ee.Reducer.mean(),
    "geometry": areaOfInterest,
    "scale": projectionScale,
    "maxPixels": 1e13
  }) \
  .get(chartIdentifiers["post_event"])

# Create histograms.
bands = list(chartIdentifiers.values())

vvHistograms = ee.Dictionary.fromLists(bands, [calculate_histogram(vvRaster, band, areaOfInterest) for band in bands])
vhHistograms = ee.Dictionary.fromLists(bands, [calculate_histogram(vhRaster, band, areaOfInterest) for band in bands])

Rename histogram keys

In [ ]:
vvHistograms = vvHistograms.rename(list(chartIdentifiers.values()), list(chartIdentifiers.keys())).getInfo()

In [ ]:
vhHistograms = vhHistograms.rename(list(chartIdentifiers.values()), list(chartIdentifiers.keys())).getInfo()

In [ ]:
# Construct output chart paths.
plotIdentifier = "_".join(list(chartIdentifiers.values()))
vvDestinationPath = os.path.join(destinationFolder, "histograms", f"{plotIdentifier}_VV")
vhDestinationPath = os.path.join(destinationFolder, "histograms", f"{plotIdentifier}_VH")

# Plot histograms.
plot_histograms(vvHistograms, "VV Histograms", vvDestinationPath)
plot_histograms(vhHistograms, "VH Histograms", vhDestinationPath)

# **8) Console**

In [ ]:
print(pixelsCount)

In [ ]:
print("*Mean values*", {
  "pre VV": prevvMean.getInfo(),
  "post VV": postvvMean.getInfo(),
  "pre VH": prevhMean.getInfo(),
  "post VH": postvhMean.getInfo()
})

Acquisition Parameters Comparison

In [ ]:
print(preEventRaster.toDictionary().getInfo())
print(postEventRaster.toDictionary().getInfo())

# **9) Map Visualization**

In [ ]:
Map = geemap.Map()

Map.centerObject(areaOfInterest)

Map.addLayer(preEventRaster, sarReddishVisualization, "rasters: S1-GRD (pre-red)")
Map.addLayer(postEventRaster, sarReddishVisualization, "rasters: S1-GRD (post-red)")

Map.addLayer(preEventRaster, sarBluishVisualization, "rasters: S1-GRD (pre-blue)")
Map.addLayer(postEventRaster, sarBluishVisualization, "rasters: S1-GRD (post-blue)")

compositeRaster = ee.Image.cat([
  preEventRaster.select("VH"), postEventRaster.select("VH"), preEventRaster.select("VH")
])

Map.addLayer(compositeRaster, {min: -25, max: -8}, "change composite");

Map.addLayer(VVVVD, {min: -10, max: 10}, "rasters: VVVVD")
Map.addLayer(VHVHD, {min: -10, max: 10}, "rasters: VHVHD")
Map.addLayer(VVVHD, {min: -8, max: 12}, "rasters: VVVHD")

Map.addLayer(VVVVQ, {min: 0, max: 3.5}, "rasters: VVVVQ")
Map.addLayer(VHVHQ, {min: 0, max: 3.5}, "rasters: VHVHQ")
Map.addLayer(VVVHQ, {min: 0, max: 3.5}, "rasters: VVVHQ")

Map.addLayer(postNDPI, {min: -1, max: 1}, "POST_NDPI")
Map.addLayer(preNDPI, {min: -1, max: 1}, "PRE_NDPI")
Map.addLayer(NDPID, {min: -1, max: 1}, "NDPID")
Map.addLayer(NDPIQ, {min: -1, max: 1}, "NDPIQ")

Map.addLayer(ring, {"color": "white"}, "vectors: area")

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

-End of Notebook-